케라스와 넘파이 패키지를 import합니다.

In [1]:
import tensorflow as tf
import numpy as np

x 가 모델에 입력이 되면 y = 2*x+1 가 출력되는 모델을 만들것입니다. 이러한 관계를 만족하는 x,y 쌍의 데이터로 학습을시켜 모델의 웨이트값 들을 조정할것입니다. 그러기위해서 y = 2*x + 1 의 관계를 만족하는 트레이닝에 사용할 데이터를 만듭니다. x_train 은 신공망에 들어가는 입력이고 y_train 은 신경망에서 출력되는 출력데이터입니다.

In [2]:
x_train = np.array( [[0],[1]] )
y_train = x_train * 2 + 1
print('x_train')
print(x_train)
print('y_train')
print(y_train)
x_test = np.array([[2],[3]])
y_test = x_test * 2 + 1
print('x_test')
print(x_test)
print('y_test')
print(y_test)

x_train
[[0]
 [1]]
y_train
[[1]
 [3]]
x_test
[[2]
 [3]]
y_test
[[5]
 [7]]


케라스의 인공 신경망 모델을 만드는 방법은 keras.models.Sequential 순차 모델을 사용하는 방법과 keras.models.Model 의 functional API 를 사용해 만드는 방법 두가지가 있습니다. 순차모델은 레이어를 순차적으로 추가해나가는 방법으로 간단하고 쉽게 모델을 만들 수 있습니다. Functional API 를 사용해 만드는 방법을 사용하면 더 다양한 모델을 만들 수 있습니다. 여기서는 간단히 모델을 만들 수 있는 순차방식을 사용하겠습니다.

In [3]:
model = tf.keras.models.Sequential()
print( type(model) )

<class 'tensorflow.python.keras.engine.sequential.Sequential'>


케라스의 층, 즉 레이어에는 여러 종류가 있는데 여기서는 전의 레이어와 현재레이어의 모든 유닛이 다 연결이 되어있는 keras.layers.Dense 레이어를 사용하겠습니다. 첫번째 인자는 레이어안에 있는 유닛의 수이고, 두번째 인자는 입력 자료의 차원 모양입니다. 여기서는 유닛이 하나인 레이어를 만들겠습니다. 첫번째 레이어기 때문에 입력데이터의 모양을 알려줘야합니다. 입력은 x 수 하나이므로 shape=(1,) 를 넣어줍니다. 차원이 하나이고 그 차원쪽으로 크기가 1이라는 의미입니다. 첫 레이어 다음 부터는 입력 모양이 자동으로 유추되므로 직접 넣어줄 필요가 없습니다.

add 함수를 사용하여 만든 레이어를 추가합니다. 괄호안에 커서를 위치시키고 Shift-Tab 을 누르면 함수의 시그너처를 볼수있습니다.

In [4]:
model.add( tf.keras.layers.Dense( 1, input_shape=(1,) ) )

summary 함수를 사용하여 만든 모델의 요약정보를 살펴봅니다. 트레이닝 데이터를 여러개 동시에 학습을 시키기 때문에 Output Shape 의 차원수는 input_shape 으로 지정한것보다 하나가 더 많고 몇개를 동시에 학습시킬것인지는 지금 알수 없으므로 None 으로 표시됩니다. 레이어에 유닛이 하나이고 웨이트값과 바이어스 값이 있으므로 Param 의 갯수는 2가 됩니다. 전체 파라메터의 갯수도 레이어가 하나이기때문에 2 가 되고 학습시킬 수 있는 값들이므로 Trainable params 의 수도 2 가 표시됩니다.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


학습시키기 전에 predict 함수를 사용해 모델에 테스트 데이터를 입력시켜 예측값을 계산해보면 예측값과 원하는 목표값이 많이 다른것을 알 수 있습니다. 출력되는 데이터의 첫번째 차원은 동시에 계산한 샘플 수에 해당하는 차원입니다.

In [6]:
y_predict = model.predict( x_test )
print( y_predict )
print( y_test )

[[0.04727077]
 [0.07090616]]
[[5]
 [7]]


모델을 학습시키기 전에 compile 함수를 사용해서 학습시키기전에 준비 작업을 합니다. 학습하기전에는 꼭 compile 을 해줘야 합니다. 첫번째 인자는 optimizer 즉 최적화방법이고 두번째 인자는 예측값과 실제 원하는 값의 차이를 정의하는 loss function 입니다. 여기서 사용하는 optimiser 는 'SGD' 로 Stochastic Gradient Descent 의 약자입니다. 기울기를 사용해서 loss function 값이 가장 작은 곳을 찾아가는 방법입니다. 로스 함수인 'mse' 는 mean squared error 의 약자입니다. 에러를 제곱하고 평균낸 값입니다.

In [7]:
model.compile( optimizer = 'SGD', loss = 'mse' )

fit 함수를 사용하여 웨이트들을 학습시킵니다. 첫번째 인수는 입력 학습 데이터의 모음, 두번째 인수는 출력 학습데이터의 모음입니다. batch_size 는 한번에 학습할 샘플의 수입니다. epochs 는 전체 학습데이터를 몇번 반복해서 학습시킬것인지 정합니다. verbose = 0 은 학습하는 동안 중간 보고를 하지말고 조용히 있어달라는 뜻입니다.

In [8]:
history = model.fit( x_train, y_train, batch_size=2, epochs=1000, verbose=0 )

다시한번 테스트 데이터를 사용해 모델이 예측한 데이터와 실제 데이터를 비교해봅니다. 학습하기전보다 예측값과 실제 값이 비슷해져있는것을 확인할 수 있습니다.

In [9]:
y_predict = model.predict( x_test )
print( y_predict )
print( y_test )

[[4.9704213]
 [6.949018 ]]
[[5]
 [7]]
